# Vibration Analysis Data Science Agent

## AI Agents Intensive Course - Capstone Project


**Problem Statement**: Manual exploratory data analysis on industrial vibration data is time-consuming. Data scientists working with condition monitoring systems need to iteratively apply various time-domain and frequency-domain techniques to understand equipment health patterns.

**Solution**: This notebook demonstrates a **Vibration Analysis EDA Agent** - a multi-agent system built with Google's Agent Development Kit (ADK) that helps data scientists interactively explore wind turbine drivetrain vibration data. The agent applies domain-specific signal processing techniques including statistical analysis, spectral analysis, and visualization.

### Key Concepts Demonstrated

This project demonstrates the following concepts from the course:

1. **Multi-Agent System** - Orchestrator agent delegating to specialized sub-agents
2. **Custom Tools** - Domain-specific vibration analysis functions
3. **Sessions & State Management** - Maintaining loaded data and analysis results across conversation turns
4. **Observability (Logging)** - Tracking agent decisions and tool executions

### Agent Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                    Orchestrator Agent                       │
│         (Routes requests to specialized agents)             │
└─────────────────────┬───────────────────────────────────────┘
                      │
        ┌─────────────┼─────────────┐
        ▼             ▼             ▼
┌───────────────┐ ┌───────────────┐ ┌───────────────┐
│  Time Domain  │ │   Frequency   │ │   Plotting    │
│    Agent      │ │ Domain Agent  │ │    Agent      │
├───────────────┤ ├───────────────┤ ├───────────────┤
│ • Statistics  │ │ • FFT         │ │ • Time plots  │
│ • RMS         │ │ • Welch PSD   │ │ • FFT plots   │
│ • Kurtosis    │ │ • Spectrogram │ │ • PSD plots   │
│ • Crest Factor│ │ • Envelope    │ │ • Trend plots │
└───────────────┘ └───────────────┘ └───────────────┘
```

## 1. Setup and Installation

Install required packages and configure the environment.

In [ ]:
# Install Google ADK and dependencies
!pip install -q google-adk>=1.0.0 numpy pandas scipy matplotlib nest_asyncio

In [ ]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy.fft import fft, fftfreq
from scipy.signal import hilbert, welch, spectrogram
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from typing import Optional
import json
import asyncio
import nest_asyncio

# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

# Google ADK imports
from google.adk.agents import Agent
from google.adk.runners import Runner  # Changed from InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.genai import types

print("All packages imported successfully!")

## 2. Synthetic Data Generation

Generate realistic synthetic vibration data from wind turbine drivetrains. The dataset includes:
- 20 wind turbines
- 2 measurement points: Main Bearing (MB) and Generator Front-End Bearing (GFEB)
- Daily measurements for January 2025
- 1-second waveforms at 1000 Hz sampling rate
- Operational metadata: rotor speed, generator speed, actual power

In [ ]:
def generate_vibration_waveform(fs: int = 1000, duration: float = 1.0, 
                                 rotor_rpm: float = 15.0, gen_rpm: float = 1500.0,
                                 base_amplitude: float = 1.0, noise_level: float = 0.3,
                                 seed: Optional[int] = None) -> np.ndarray:
    """
    Generate a synthetic vibration waveform with realistic characteristics.
    
    Components:
    - Shaft rotation harmonics (1X, 2X, 3X)
    - Bearing characteristic frequencies (simulated)
    - Random noise
    """
    if seed is not None:
        np.random.seed(seed)
    
    t = np.linspace(0, duration, int(fs * duration), endpoint=False)
    
    # Shaft frequencies
    f_rotor = rotor_rpm / 60  # Hz
    f_gen = gen_rpm / 60  # Hz
    
    # Build signal with harmonics
    signal_out = np.zeros_like(t)
    
    # Rotor harmonics (1X, 2X, 3X)
    for harmonic in [1, 2, 3]:
        amp = base_amplitude / harmonic
        signal_out += amp * np.sin(2 * np.pi * harmonic * f_rotor * t + np.random.uniform(0, 2*np.pi))
    
    # Generator harmonics (scaled down, higher frequency)
    for harmonic in [1, 2]:
        amp = base_amplitude * 0.3 / harmonic
        signal_out += amp * np.sin(2 * np.pi * harmonic * f_gen * t + np.random.uniform(0, 2*np.pi))
    
    # Simulated bearing frequencies (BPFO-like)
    f_bearing = f_rotor * 7.5  # Typical ball pass frequency
    signal_out += base_amplitude * 0.15 * np.sin(2 * np.pi * f_bearing * t)
    
    # Add noise
    signal_out += noise_level * np.random.randn(len(t))
    
    return signal_out


def generate_dataset(n_turbines: int = 20, start_date: str = "2025-01-01", 
                     n_days: int = 31, fs: int = 1000) -> pd.DataFrame:
    """
    Generate complete synthetic dataset for wind turbine vibration analysis.
    """
    measurement_points = ['MB', 'GFEB']  # Main Bearing, Generator Front-End Bearing
    start = datetime.strptime(start_date, "%Y-%m-%d")
    
    records = []
    
    for turbine_id in range(1, n_turbines + 1):
        # Each turbine has slightly different baseline characteristics
        turbine_base_amp = 0.8 + 0.4 * np.random.random()
        
        for day_offset in range(n_days):
            date = start + timedelta(days=day_offset)
            
            # Operational conditions vary with wind/time
            rotor_rpm = 10 + 10 * np.random.random()  # 10-20 RPM typical
            gen_rpm = rotor_rpm * 100  # Gearbox ratio ~100:1
            power_kw = (rotor_rpm / 20) ** 3 * 2000 * (0.8 + 0.2 * np.random.random())  # Cubic relation
            
            for mp in measurement_points:
                # Different characteristics per measurement point
                if mp == 'MB':
                    amp_factor = 1.0
                    noise_factor = 0.3
                else:  # GFEB - typically higher frequency content
                    amp_factor = 0.7
                    noise_factor = 0.4
                
                seed = turbine_id * 10000 + day_offset * 100 + (0 if mp == 'MB' else 1)
                waveform = generate_vibration_waveform(
                    fs=fs,
                    rotor_rpm=rotor_rpm,
                    gen_rpm=gen_rpm,
                    base_amplitude=turbine_base_amp * amp_factor,
                    noise_level=noise_factor,
                    seed=seed
                )
                
                records.append({
                    'turbine_id': f'WT{turbine_id:02d}',
                    'date': date.strftime("%Y-%m-%d"),
                    'measurement_point': mp,
                    'rotor_rpm': round(rotor_rpm, 2),
                    'generator_rpm': round(gen_rpm, 2),
                    'power_kw': round(power_kw, 2),
                    'fs': fs,
                    'waveform': waveform.tolist()
                })
    
    return pd.DataFrame(records)


# Generate the dataset
print("Generating synthetic vibration dataset...")
df_vibration = generate_dataset(n_turbines=20, n_days=31)
print(f"Dataset shape: {df_vibration.shape}")
print(f"Columns: {df_vibration.columns.tolist()}")
print(f"\nSample record (without waveform):")
print(df_vibration.drop(columns=['waveform']).head())

## 3. Analysis Tool Functions

Define the domain-specific tools that our agents will use for vibration analysis.

In [ ]:
# Global state to store loaded data and results
STATE = {
    'dataset': df_vibration,
    'current_waveform': None,
    'current_metadata': None,
    'analysis_results': {},
    'fs': 1000
}


# ============== Data Loading Tools ==============

def list_available_data() -> dict:
    """
    List available turbines, measurement points, and date range in the dataset.
    Use this to understand what data is available for analysis.
    """
    df = STATE['dataset']
    return {
        'turbines': sorted(df['turbine_id'].unique().tolist()),
        'measurement_points': df['measurement_point'].unique().tolist(),
        'date_range': {'start': df['date'].min(), 'end': df['date'].max()},
        'total_records': len(df)
    }


def load_waveform(turbine_id: str, measurement_point: str, date: str) -> dict:
    """
    Load a specific vibration waveform for analysis.
    
    Args:
        turbine_id: Turbine identifier (e.g., 'WT01')
        measurement_point: 'MB' (Main Bearing) or 'GFEB' (Generator Front-End Bearing)
        date: Date string in YYYY-MM-DD format
    
    Returns:
        Dictionary with waveform info and operational metadata
    """
    df = STATE['dataset']
    mask = (df['turbine_id'] == turbine_id) & \
           (df['measurement_point'] == measurement_point) & \
           (df['date'] == date)
    
    if mask.sum() == 0:
        return {'error': f'No data found for {turbine_id}, {measurement_point}, {date}'}
    
    record = df[mask].iloc[0]
    waveform = np.array(record['waveform'])
    
    STATE['current_waveform'] = waveform
    STATE['current_metadata'] = {
        'turbine_id': turbine_id,
        'measurement_point': measurement_point,
        'date': date,
        'rotor_rpm': record['rotor_rpm'],
        'generator_rpm': record['generator_rpm'],
        'power_kw': record['power_kw'],
        'fs': record['fs']
    }
    STATE['fs'] = record['fs']
    STATE['analysis_results'] = {}  # Clear previous results
    
    return {
        'status': 'success',
        'message': f'Loaded waveform for {turbine_id} - {measurement_point} on {date}',
        'samples': len(waveform),
        'duration_sec': len(waveform) / record['fs'],
        'operational_data': {
            'rotor_rpm': record['rotor_rpm'],
            'generator_rpm': record['generator_rpm'],
            'power_kw': record['power_kw']
        }
    }

In [ ]:
# ============== Time Domain Analysis Tools ==============

def compute_time_domain_statistics() -> dict:
    """
    Compute comprehensive time-domain statistics for the loaded waveform.
    
    Returns:
        Dictionary containing: mean, std, rms, peak, peak_to_peak, 
        crest_factor, kurtosis, skewness
    """
    if STATE['current_waveform'] is None:
        return {'error': 'No waveform loaded. Use load_waveform first.'}
    
    x = STATE['current_waveform']
    
    # Basic statistics
    mean_val = float(np.mean(x))
    std_val = float(np.std(x))
    rms_val = float(np.sqrt(np.mean(x**2)))
    
    # Peak values
    peak_val = float(np.max(np.abs(x)))
    peak_to_peak = float(np.max(x) - np.min(x))
    
    # Crest factor (peak / RMS)
    crest_factor = float(peak_val / rms_val) if rms_val > 0 else 0
    
    # Higher-order statistics
    from scipy.stats import kurtosis, skew
    kurtosis_val = float(kurtosis(x))
    skewness_val = float(skew(x))
    
    results = {
        'mean': round(mean_val, 6),
        'std': round(std_val, 6),
        'rms': round(rms_val, 6),
        'peak': round(peak_val, 6),
        'peak_to_peak': round(peak_to_peak, 6),
        'crest_factor': round(crest_factor, 4),
        'kurtosis': round(kurtosis_val, 4),
        'skewness': round(skewness_val, 4)
    }
    
    STATE['analysis_results']['time_domain_stats'] = results
    return results


def compute_rms_trend(turbine_id: str, measurement_point: str, 
                      start_date: str, end_date: str) -> dict:
    """
    Compute RMS trend over a date range for a specific turbine and measurement point.
    
    Args:
        turbine_id: Turbine identifier (e.g., 'WT01')
        measurement_point: 'MB' or 'GFEB'
        start_date: Start date (YYYY-MM-DD)
        end_date: End date (YYYY-MM-DD)
    
    Returns:
        Dictionary with dates and corresponding RMS values
    """
    df = STATE['dataset']
    mask = (df['turbine_id'] == turbine_id) & \
           (df['measurement_point'] == measurement_point) & \
           (df['date'] >= start_date) & \
           (df['date'] <= end_date)
    
    filtered = df[mask].sort_values('date')
    
    if len(filtered) == 0:
        return {'error': 'No data found for the specified parameters'}
    
    dates = []
    rms_values = []
    
    for _, row in filtered.iterrows():
        waveform = np.array(row['waveform'])
        rms = float(np.sqrt(np.mean(waveform**2)))
        dates.append(row['date'])
        rms_values.append(round(rms, 6))
    
    results = {
        'turbine_id': turbine_id,
        'measurement_point': measurement_point,
        'dates': dates,
        'rms_values': rms_values,
        'mean_rms': round(np.mean(rms_values), 6),
        'std_rms': round(np.std(rms_values), 6),
        'trend_direction': 'increasing' if rms_values[-1] > rms_values[0] else 'decreasing'
    }
    
    STATE['analysis_results']['rms_trend'] = results
    return results

In [ ]:
# ============== Frequency Domain Analysis Tools ==============

def compute_fft_spectrum() -> dict:
    """
    Compute FFT spectrum of the loaded waveform.
    
    Returns:
        Dictionary with frequency array, amplitude spectrum, and dominant frequencies
    """
    if STATE['current_waveform'] is None:
        return {'error': 'No waveform loaded. Use load_waveform first.'}
    
    x = STATE['current_waveform']
    fs = STATE['fs']
    n = len(x)
    
    # Compute FFT
    fft_vals = fft(x)
    freqs = fftfreq(n, 1/fs)
    
    # Take positive frequencies only
    pos_mask = freqs >= 0
    freqs_pos = freqs[pos_mask]
    amplitude = np.abs(fft_vals[pos_mask]) * 2 / n
    
    # Find dominant frequencies (peaks)
    from scipy.signal import find_peaks
    peaks, _ = find_peaks(amplitude, height=np.max(amplitude) * 0.1)
    dominant_freqs = [(float(freqs_pos[p]), float(amplitude[p])) for p in peaks[:10]]
    dominant_freqs.sort(key=lambda x: x[1], reverse=True)
    
    results = {
        'frequencies': freqs_pos.tolist(),
        'amplitude': amplitude.tolist(),
        'dominant_frequencies': [{'freq_hz': f, 'amplitude': a} for f, a in dominant_freqs[:5]],
        'freq_resolution': float(fs / n)
    }
    
    STATE['analysis_results']['fft'] = results
    return {
        'status': 'success',
        'dominant_frequencies': results['dominant_frequencies'],
        'freq_resolution': results['freq_resolution'],
        'message': 'FFT computed. Use plotting agent to visualize.'
    }


def compute_welch_psd(nperseg: int = 256) -> dict:
    """
    Compute Power Spectral Density using Welch's method.
    
    Args:
        nperseg: Length of each segment for Welch's method
    
    Returns:
        Dictionary with frequencies and PSD values
    """
    if STATE['current_waveform'] is None:
        return {'error': 'No waveform loaded. Use load_waveform first.'}
    
    x = STATE['current_waveform']
    fs = STATE['fs']
    
    freqs, psd = welch(x, fs=fs, nperseg=min(nperseg, len(x)//2))
    
    # Find peak frequency
    peak_idx = np.argmax(psd)
    
    results = {
        'frequencies': freqs.tolist(),
        'psd': psd.tolist(),
        'peak_frequency_hz': float(freqs[peak_idx]),
        'peak_psd': float(psd[peak_idx]),
        'total_power': float(np.trapz(psd, freqs))
    }
    
    STATE['analysis_results']['welch'] = results
    return {
        'status': 'success',
        'peak_frequency_hz': results['peak_frequency_hz'],
        'total_power': round(results['total_power'], 6),
        'message': 'Welch PSD computed. Use plotting agent to visualize.'
    }


def compute_spectrogram(nperseg: int = 128) -> dict:
    """
    Compute spectrogram (time-frequency representation) of the loaded waveform.
    
    Args:
        nperseg: Length of each segment
    
    Returns:
        Dictionary with time, frequency, and spectrogram matrix info
    """
    if STATE['current_waveform'] is None:
        return {'error': 'No waveform loaded. Use load_waveform first.'}
    
    x = STATE['current_waveform']
    fs = STATE['fs']
    
    f, t, Sxx = spectrogram(x, fs=fs, nperseg=min(nperseg, len(x)//4))
    
    results = {
        'frequencies': f.tolist(),
        'times': t.tolist(),
        'spectrogram': Sxx.tolist(),
        'freq_bins': len(f),
        'time_bins': len(t)
    }
    
    STATE['analysis_results']['spectrogram'] = results
    return {
        'status': 'success',
        'freq_bins': len(f),
        'time_bins': len(t),
        'message': 'Spectrogram computed. Use plotting agent to visualize.'
    }


def compute_envelope_spectrum() -> dict:
    """
    Compute envelope spectrum using Hilbert transform.
    Useful for detecting bearing fault frequencies.
    
    Returns:
        Dictionary with envelope spectrum data
    """
    if STATE['current_waveform'] is None:
        return {'error': 'No waveform loaded. Use load_waveform first.'}
    
    x = STATE['current_waveform']
    fs = STATE['fs']
    
    # Compute analytic signal using Hilbert transform
    analytic_signal = hilbert(x)
    envelope = np.abs(analytic_signal)
    
    # Remove DC component from envelope
    envelope = envelope - np.mean(envelope)
    
    # Compute FFT of envelope
    n = len(envelope)
    fft_env = fft(envelope)
    freqs = fftfreq(n, 1/fs)
    
    pos_mask = freqs >= 0
    freqs_pos = freqs[pos_mask]
    amplitude = np.abs(fft_env[pos_mask]) * 2 / n
    
    # Find peaks in envelope spectrum
    from scipy.signal import find_peaks
    peaks, _ = find_peaks(amplitude, height=np.max(amplitude) * 0.1)
    dominant_freqs = [(float(freqs_pos[p]), float(amplitude[p])) for p in peaks[:10]]
    dominant_freqs.sort(key=lambda x: x[1], reverse=True)
    
    results = {
        'frequencies': freqs_pos.tolist(),
        'envelope_amplitude': amplitude.tolist(),
        'envelope_time': envelope.tolist(),
        'dominant_frequencies': [{'freq_hz': f, 'amplitude': a} for f, a in dominant_freqs[:5]]
    }
    
    STATE['analysis_results']['envelope'] = results
    return {
        'status': 'success',
        'dominant_frequencies': results['dominant_frequencies'],
        'message': 'Envelope spectrum computed. Use plotting agent to visualize.'
    }

In [ ]:
# ============== Plotting Tools ==============

def plot_time_waveform() -> dict:
    """
    Plot the time-domain waveform of the loaded signal.
    """
    if STATE['current_waveform'] is None:
        return {'error': 'No waveform loaded. Use load_waveform first.'}
    
    x = STATE['current_waveform']
    fs = STATE['fs']
    t = np.arange(len(x)) / fs
    meta = STATE['current_metadata']
    
    plt.figure(figsize=(12, 4))
    plt.plot(t, x, 'b-', linewidth=0.5)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.title(f"Time Waveform - {meta['turbine_id']} | {meta['measurement_point']} | {meta['date']}")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return {'status': 'success', 'message': 'Time waveform plot generated'}


def plot_fft_spectrum() -> dict:
    """
    Plot the FFT amplitude spectrum. Must compute FFT first.
    """
    if 'fft' not in STATE['analysis_results']:
        return {'error': 'FFT not computed. Run compute_fft_spectrum first.'}
    
    results = STATE['analysis_results']['fft']
    meta = STATE['current_metadata']
    
    freqs = np.array(results['frequencies'])
    amplitude = np.array(results['amplitude'])
    
    plt.figure(figsize=(12, 4))
    plt.plot(freqs, amplitude, 'b-', linewidth=0.5)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')
    plt.title(f"FFT Spectrum - {meta['turbine_id']} | {meta['measurement_point']} | {meta['date']}")
    plt.xlim([0, STATE['fs']/2])
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return {'status': 'success', 'message': 'FFT spectrum plot generated'}


def plot_welch_psd() -> dict:
    """
    Plot the Power Spectral Density (Welch). Must compute Welch PSD first.
    """
    if 'welch' not in STATE['analysis_results']:
        return {'error': 'Welch PSD not computed. Run compute_welch_psd first.'}
    
    results = STATE['analysis_results']['welch']
    meta = STATE['current_metadata']
    
    freqs = np.array(results['frequencies'])
    psd = np.array(results['psd'])
    
    plt.figure(figsize=(12, 4))
    plt.semilogy(freqs, psd, 'b-', linewidth=1)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('PSD')
    plt.title(f"Power Spectral Density (Welch) - {meta['turbine_id']} | {meta['measurement_point']} | {meta['date']}")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return {'status': 'success', 'message': 'Welch PSD plot generated'}


def plot_spectrogram() -> dict:
    """
    Plot the spectrogram. Must compute spectrogram first.
    """
    if 'spectrogram' not in STATE['analysis_results']:
        return {'error': 'Spectrogram not computed. Run compute_spectrogram first.'}
    
    results = STATE['analysis_results']['spectrogram']
    meta = STATE['current_metadata']
    
    f = np.array(results['frequencies'])
    t = np.array(results['times'])
    Sxx = np.array(results['spectrogram'])
    
    plt.figure(figsize=(12, 5))
    plt.pcolormesh(t, f, 10 * np.log10(Sxx + 1e-10), shading='gouraud', cmap='viridis')
    plt.colorbar(label='Power/Frequency (dB/Hz)')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.title(f"Spectrogram - {meta['turbine_id']} | {meta['measurement_point']} | {meta['date']}")
    plt.tight_layout()
    plt.show()
    
    return {'status': 'success', 'message': 'Spectrogram plot generated'}


def plot_envelope_spectrum() -> dict:
    """
    Plot the envelope spectrum. Must compute envelope spectrum first.
    """
    if 'envelope' not in STATE['analysis_results']:
        return {'error': 'Envelope spectrum not computed. Run compute_envelope_spectrum first.'}
    
    results = STATE['analysis_results']['envelope']
    meta = STATE['current_metadata']
    
    freqs = np.array(results['frequencies'])
    amplitude = np.array(results['envelope_amplitude'])
    
    plt.figure(figsize=(12, 4))
    plt.plot(freqs, amplitude, 'b-', linewidth=0.5)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Envelope Amplitude')
    plt.title(f"Envelope Spectrum - {meta['turbine_id']} | {meta['measurement_point']} | {meta['date']}")
    plt.xlim([0, 100])  # Focus on lower frequencies for bearing analysis
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return {'status': 'success', 'message': 'Envelope spectrum plot generated'}


def plot_rms_trend() -> dict:
    """
    Plot the RMS trend over time. Must compute RMS trend first.
    """
    if 'rms_trend' not in STATE['analysis_results']:
        return {'error': 'RMS trend not computed. Run compute_rms_trend first.'}
    
    results = STATE['analysis_results']['rms_trend']
    
    dates = pd.to_datetime(results['dates'])
    rms_values = results['rms_values']
    
    plt.figure(figsize=(12, 4))
    plt.plot(dates, rms_values, 'b-o', markersize=4)
    plt.axhline(y=results['mean_rms'], color='r', linestyle='--', label=f"Mean: {results['mean_rms']:.4f}")
    plt.fill_between(dates, 
                     results['mean_rms'] - results['std_rms'], 
                     results['mean_rms'] + results['std_rms'], 
                     alpha=0.2, color='red', label='±1 Std')
    plt.xlabel('Date')
    plt.ylabel('RMS')
    plt.title(f"RMS Trend - {results['turbine_id']} | {results['measurement_point']}")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    return {'status': 'success', 'message': 'RMS trend plot generated'}

## 4. Agent Definitions

Define the multi-agent system using Google ADK.

In [ ]:
# Time Domain Analysis Agent
time_domain_agent = Agent(
    name="time_domain_agent",
    model="gemini-2.0-flash",
    description="Specialist agent for time-domain vibration analysis including statistical metrics and energy-based parameters.",
    instruction="""You are a time-domain vibration analysis specialist. Your role is to:
    
1. Compute statistical parameters: mean, standard deviation, RMS, peak values
2. Calculate energy-based metrics: crest factor, kurtosis, skewness
3. Analyze RMS trends over time to detect degradation patterns

When asked to analyze a signal:
- First ensure a waveform is loaded using load_waveform
- Use compute_time_domain_statistics for single waveform analysis
- Use compute_rms_trend for temporal trend analysis

Provide clear interpretations of the results:
- High kurtosis (>3) may indicate impulsive events or bearing damage
- High crest factor indicates presence of peaks/impacts
- RMS trend increases may indicate developing faults

Always explain what each metric means in the context of vibration analysis.""",
    tools=[load_waveform, list_available_data, compute_time_domain_statistics, compute_rms_trend]
)

print("Time Domain Agent created successfully!")

In [ ]:
# Frequency Domain Analysis Agent
frequency_domain_agent = Agent(
    name="frequency_domain_agent",
    model="gemini-2.0-flash",
    description="Specialist agent for frequency-domain vibration analysis including spectral and envelope analysis.",
    instruction="""You are a frequency-domain vibration analysis specialist. Your role is to:

1. Compute FFT spectrum to identify dominant frequencies
2. Calculate Power Spectral Density using Welch's method for smoother spectral estimates
3. Generate spectrograms for time-frequency analysis
4. Perform envelope analysis using Hilbert transform for bearing fault detection

When analyzing frequency content:
- Use compute_fft_spectrum for basic spectral analysis
- Use compute_welch_psd for energy distribution analysis
- Use compute_spectrogram for time-varying frequency content
- Use compute_envelope_spectrum for bearing fault frequency identification

Key frequency relationships to consider:
- Shaft rotation frequency = RPM / 60
- Harmonics (2X, 3X, etc.) indicate misalignment or looseness
- Bearing frequencies typically appear in envelope spectrum

Always relate dominant frequencies to the operational speed when interpreting results.""",
    tools=[load_waveform, list_available_data, compute_fft_spectrum, compute_welch_psd, 
           compute_spectrogram, compute_envelope_spectrum]
)

print("Frequency Domain Agent created successfully!")

In [ ]:
# Plotting Agent
plotting_agent = Agent(
    name="plotting_agent",
    model="gemini-2.0-flash",
    description="Specialist agent for creating visualizations of vibration analysis results.",
    instruction="""You are a visualization specialist for vibration data. Your role is to create clear and informative plots.

Available plotting functions:
- plot_time_waveform: Show the raw time-domain signal
- plot_fft_spectrum: Display FFT amplitude spectrum (requires FFT computation first)
- plot_welch_psd: Show Power Spectral Density plot (requires Welch computation first)
- plot_spectrogram: Display time-frequency representation (requires spectrogram computation first)
- plot_envelope_spectrum: Show envelope spectrum for bearing analysis (requires envelope computation first)
- plot_rms_trend: Display RMS trend over time (requires RMS trend computation first)

Important notes:
- Ensure the required analysis has been performed before attempting to plot
- If a plot fails because analysis is missing, inform the user which analysis needs to be done first
- For comprehensive analysis, recommend plotting multiple views of the data

When asked to visualize, create the appropriate plots and describe what they show.""",
    tools=[plot_time_waveform, plot_fft_spectrum, plot_welch_psd, 
           plot_spectrogram, plot_envelope_spectrum, plot_rms_trend]
)

print("Plotting Agent created successfully!")

In [ ]:
# Root Orchestrator Agent
orchestrator_agent = Agent(
    name="vibration_analysis_orchestrator",
    model="gemini-2.0-flash",
    description="Main orchestrator for vibration analysis tasks, delegating to specialized sub-agents.",
    instruction="""You are the main orchestrator for a vibration analysis system for wind turbine drivetrains.
    
You coordinate three specialist agents:
1. **time_domain_agent**: For statistical analysis, RMS computation, kurtosis, crest factor, and trend analysis
2. **frequency_domain_agent**: For FFT, PSD, spectrogram, and envelope spectrum analysis
3. **plotting_agent**: For creating visualizations of any analysis results

Your workflow:
1. Understand the user's analysis request
2. Delegate to the appropriate specialist agent(s)
3. For visualizations, ensure the required analysis is done first, then delegate to plotting_agent
4. Synthesize results and provide clear explanations

Available data:
- 20 wind turbines (WT01-WT20)
- 2 measurement points: MB (Main Bearing), GFEB (Generator Front-End Bearing)
- Date range: January 2025
- Operational data: rotor RPM, generator RPM, power (kW)

When users ask general questions about the data or what's available, use the list_available_data tool.
For any analysis, first load the appropriate waveform using load_waveform.

Guide users through a logical analysis workflow and explain the significance of findings.""",
    tools=[list_available_data, load_waveform],
    sub_agents=[time_domain_agent, frequency_domain_agent, plotting_agent]
)

print("Orchestrator Agent created successfully!")
print("\nMulti-agent system ready with:")
print("  - Orchestrator (root agent)")
print("  - Time Domain Agent")
print("  - Frequency Domain Agent")
print("  - Plotting Agent")

## 5. Session Setup and Runner

Configure the session service for state management and create the runner.

In [ ]:
# Session configuration
APP_NAME = "vibration_analysis"
USER_ID = "data_scientist_01"
SESSION_ID = "session_001"

# Create session service for state management
session_service = InMemorySessionService()

# Create session first
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

# Create runner with the orchestrator agent
runner = Runner(
    agent=orchestrator_agent,
    app_name=APP_NAME,
    session_service=session_service
)

print(f"Session initialized:")
print(f"  App: {APP_NAME}")
print(f"  User: {USER_ID}")
print(f"  Session: {SESSION_ID}")

In [ ]:
async def chat(user_message: str) -> str:
    """
    Send a message to the agent and get a response.
    Includes logging for observability.
    """
    print(f"\n{'='*60}")
    print(f"USER: {user_message}")
    print(f"{'='*60}")
    
    # Create the user message content
    content = types.Content(
        role="user",
        parts=[types.Part.from_text(text=user_message)]
    )
    
    final_response = ""
    
    # Run the agent and collect responses
    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=content
    ):
        # Log tool calls for observability
        if event.get_function_calls():
            for fc in event.get_function_calls():
                print(f"  [TOOL CALL] {fc.name}({fc.args})")
        
        # Collect final text response
        if event.is_final_response() and event.content and event.content.parts:
            for part in event.content.parts:
                if hasattr(part, 'text') and part.text:
                    final_response += part.text
    
    print(f"\nAGENT: {final_response}")
    return final_response


def run_chat(message: str) -> str:
    """Synchronous wrapper for the chat function."""
    return asyncio.get_event_loop().run_until_complete(chat(message))

## 6. Interactive Demo

Let's demonstrate the agent system with example queries.

In [ ]:
# Query 1: Explore available data
await chat("What data is available in the system? Show me the turbines, measurement points, and date range.")

In [ ]:
# Query 2: Load and analyze a specific waveform
await chat("Load the vibration data for turbine WT05, Main Bearing, on January 15, 2025 and compute time-domain statistics.")

In [ ]:
# Query 3: Visualize the time waveform
await chat("Plot the time waveform for the loaded signal.")

In [ ]:
# Query 4: Frequency analysis
await chat("Perform FFT analysis on the current signal and show me the dominant frequencies.")

In [ ]:
# Query 5: Plot FFT spectrum
await chat("Plot the FFT spectrum.")

In [ ]:
# Query 6: Welch PSD analysis
await chat("Calculate and plot the Power Spectral Density using Welch's method.")

In [ ]:
# Query 7: Envelope spectrum for bearing analysis
await chat("Compute the envelope spectrum using Hilbert transform and plot it. This is useful for bearing fault detection.")

In [ ]:
# Query 8: RMS trend analysis
await chat("Calculate the RMS trend for turbine WT05, Main Bearing, for the entire month of January 2025 and plot it.")

In [ ]:
# Query 9: Compare different measurement points
await chat("Load data for turbine WT05, Generator Front-End Bearing (GFEB), same date (January 15, 2025) and perform time-domain analysis. How does it compare to the Main Bearing?")

In [ ]:
# Query 10: Spectrogram analysis
await chat("Generate a spectrogram for the current signal to see the time-frequency characteristics.")

## 7. Custom Analysis Session

This section allows for interactive exploration. Modify the queries below to explore the data.

In [ ]:
# Custom query - modify as needed
await chat("Give me a comprehensive analysis of turbine WT10 Main Bearing on January 20, 2025. Include time-domain stats, FFT, and plots.")

## 8. Project Summary and Value Articulation

### What This Agent Does

The **Vibration Analysis Data Science Agent** is a multi-agent system that assists data scientists in performing exploratory data analysis on industrial vibration data from wind turbine drivetrains. It provides:

1. **Natural Language Interface**: Query the data and request analyses using conversational language instead of writing code
2. **Automated Workflow Orchestration**: The orchestrator intelligently routes requests to specialized agents
3. **Domain-Specific Analysis**: Built-in signal processing tools tailored for vibration analysis
4. **Interactive Visualization**: Generate plots on demand to understand data patterns

### Key Concepts Demonstrated

| Concept | Implementation |
|---------|---------------|
| **Multi-Agent System** | Root orchestrator delegates to time_domain_agent, frequency_domain_agent, and plotting_agent |
| **Custom Tools** | 15+ domain-specific functions for vibration analysis (RMS, FFT, Welch, Envelope, etc.) |
| **Sessions & State** | InMemorySessionService maintains conversation context and loaded data across turns |
| **Observability** | Tool call logging shows agent decisions and execution flow |

### Value Proposition

**Time Savings**: Instead of writing repetitive analysis code, data scientists can:
- Load and explore data with simple queries
- Request complex analyses in natural language
- Generate visualizations without matplotlib boilerplate
- Compare different signals and measurement points easily

**Estimated Impact**: This agent can reduce initial data exploration time by **2-4 hours per analysis session** by eliminating the need to write custom scripts for common vibration analysis tasks.

### Future Enhancements

1. **Anomaly Detection Agent**: Add ML-based fault detection capabilities
2. **Report Generation**: Automatically compile analysis results into reports
3. **Real-time Data Integration**: Connect to live sensor data streams

---

### Technical Stack

- **Framework**: Google Agent Development Kit (ADK)
- **Model**: Gemini 2.0 Flash
- **Signal Processing**: NumPy, SciPy
- **Visualization**: Matplotlib
- **Data Handling**: Pandas

---

*This notebook was developed as a capstone project for the Google AI Agents Intensive Course (November 2025).*